# Part C. Multinomial (Softmax) Logistic Regression from Scratch

##  Teoría
## Derivación del gradiente de la función de log-verosimilitud en la regresión Softmax

El objetivo es derivar el gradiente de la **log-verosimilitud** para el modelo de **regresión logística multinomial (Softmax)**. Este modelo extiende la regresión logística binaria al caso multiclase, permitiendo modelar la probabilidad de pertenencia a cada una de $K$ clases mutuamente excluyentes.

---

### Paso 0. Planteamiento general del modelo

Para un conjunto de datos $\mathcal{D}=\{(x^{(i)}, y^{(i)})\}_{i=1}^n$, donde $x^{(i)} \in \mathbb{R}^d$ y $y^{(i)} \in \{1,2,\dots,K\}$, el modelo Softmax define la probabilidad de que un individuo $i$ pertenezca a la clase $k$ como:

$$
P_{\theta}(y=k \mid x^{(i)}) \;=\;
\frac{\exp\!\big(\theta_k^\top x^{(i)}\big)}
{\sum_{j=1}^{K}\exp\!\big(\theta_j^\top x^{(i)}\big)},
$$

donde $\theta_k \in \mathbb{R}^d$ es el vector de parámetros correspondiente a la clase $k$, y $\theta=[\theta_1,\theta_2,\dots,\theta_K]$ agrupa todos los parámetros del modelo.

El objetivo del entrenamiento es **maximizar la log-verosimilitud** de los datos bajo este modelo.

---

### Paso 1. Función de log-verosimilitud

La verosimilitud del conjunto de datos es:
$$
L(\theta) \;=\; \prod_{i=1}^{n} P_{\theta}\!\big(y^{(i)} \mid x^{(i)}\big).
$$

Tomando logaritmos:
$$
\ell(\theta) \;=\; \log L(\theta)
\;=\; \sum_{i=1}^{n} \log P_{\theta}\!\big(y^{(i)} \mid x^{(i)}\big).
$$

Sustituyendo la forma Softmax:
$$
\ell(\theta) \;=\;
\sum_{i=1}^{n} \log\!\left(
\frac{\exp\!\big(\theta_{y^{(i)}}^\top x^{(i)}\big)}
{\sum_{j=1}^{K}\exp\!\big(\theta_j^\top x^{(i)}\big)}
\right).
$$

Separando numerador y denominador:
$$
\ell(\theta) \;=\;
\underbrace{\sum_{i=1}^{n}\theta_{y^{(i)}}^\top x^{(i)}}_{\text{(A)}}
\;-\;
\underbrace{\sum_{i=1}^{n}\log\!\left(\sum_{j=1}^{K}\exp\!\big(\theta_j^\top x^{(i)}\big)\right)}_{\text{(B)}}.
$$

Esta forma es conveniente: (A) es lineal en los parámetros y (B) es un log-sum-exp, derivable con la regla de la cadena.

---

### Paso 2. Derivación del término (A)

El término (A) involucra únicamente los parámetros de la clase verdadera $y^{(i)}$ de cada observación. Al derivar respecto de $\theta_\ell$:

$$
\frac{\partial \text{(A)}}{\partial \theta_\ell}
\;=\;
\sum_{i=1}^{n} \mathbf{1}\{\,y^{(i)}=\ell\,\}\; x^{(i)}.
$$

Aquí, $\mathbf{1}\{\,y^{(i)}=\ell\,\}$ vale $1$ si $y^{(i)}=\ell$, y $0$ en caso contrario.

---

### Paso 3. Derivación del término (B) — Regla de la cadena

Defínase
$$
g_i(\theta) \;=\; \log\!\left(\sum_{j=1}^{K}\exp\!\big(\theta_j^\top x^{(i)}\big)\right).
$$

Aplicando la regla de la cadena:
$$
\frac{\partial g_i}{\partial \theta_\ell}
\;=\;
\frac{1}{\sum_{j=1}^{K}\exp\!\big(\theta_j^\top x^{(i)}\big)}
\cdot
\exp\!\big(\theta_\ell^\top x^{(i)}\big)\; x^{(i)}
\;=\;
P_{\theta}(y=\ell \mid x^{(i)})\; x^{(i)}.
$$

Sumando sobre $i$ y recordando que (B) **se resta** en $\ell(\theta)$:
$$
-\frac{\partial \text{(B)}}{\partial \theta_\ell}
\;=\;
-\sum_{i=1}^{n} P_{\theta}(y=\ell \mid x^{(i)})\; x^{(i)}.
$$

---

### Paso 4. Combinación de ambos resultados

El gradiente total respecto de $\theta_\ell$ resulta:
$$
\frac{\partial \ell(\theta)}{\partial \theta_\ell}
\;=\;
\sum_{i=1}^{n}
\Big(
\mathbf{1}\{\,y^{(i)}=\ell\,\} \;-\; P_{\theta}(y=\ell \mid x^{(i)})
\Big)\; x^{(i)}.
$$

O de forma equivalente (más “visual”):
$$
\frac{\partial \ell(\theta)}{\partial \theta_\ell}
\;=\;
\sum_{i=1}^{n}
x^{(i)}
\left[
\mathbf{1}\{\,y^{(i)}=\ell\,\}
-
\frac{\exp\!\big(\theta_\ell^\top x^{(i)}\big)}
{\sum_{j=1}^{K}\exp\!\big(\theta_j^\top x^{(i)}\big)}
\right].
$$

Este resultado refleja **observado − predicho** (en probabilidad) ponderado por las características $x^{(i)}$.

---

### Paso 5. Forma matricial

Para implementación vectorizada, sea:

- $X \in \mathbb{R}^{n \times d}$: matriz de datos (filas $x^{(i)\top}$),
- $Y \in \{0,1\}^{n \times K}$: codificación one-hot de las etiquetas,
- $P = \mathrm{softmax}(X\theta) \in \mathbb{R}^{n \times K}$: probabilidades predichas.

Entonces:
$$
\nabla_{\theta}\, \ell(\theta) \;=\; X^\top (Y - P).
$$

---

### Paso 6. Interpretación del resultado

- $Y - P$ es la diferencia entre lo **observado** y lo **predicho**.
- $X^\top (Y - P)$ acumula esa diferencia ponderada por cada variable explicativa.
- Si el modelo predice bien, $Y \approx P$ y el gradiente tiende a cero (convergencia).

En la práctica se **minimiza** la NLL (negativa de la log-verosimilitud), de modo que:
$$
\nabla_{\theta} L_{\text{NLL}} \;=\; X^\top (P - Y).
$$

---

### Resultado final

$$
\boxed{
\frac{\partial \ell(\theta)}{\partial \theta_\ell}
=
\sum_{i=1}^{n}
x^{(i)}\Big[
\mathbf{1}\{\,y^{(i)}=\ell\,\}
-
P_{\theta}(y=\ell \mid x^{(i)})
\Big],
\qquad
\nabla_{\theta}\, \ell(\theta) \;=\; X^\top (Y - P).
}
$$